## 1. Importación de Librerías

In [ ]:
import neurokit2 as nk
import numpy as np
import scipy.signal as signal
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from tqdm import tqdm

## 2. Detección de Complejos QRS y Picos R

In [ ]:
from scipy.signal import butter, filtfilt, find_peaks

### 2.1 Funciones Auxiliares

In [ ]:
def moving_average(x, w):
    w = max(1, int(w))
    c = np.ones(w) / w
    return np.convolve(x, c, mode='same')


def _refine_peak(ecg, idx, half_win):
    """Refine approximate peak to the max of the raw ECG in ±half_win."""
    i0 = max(0, idx-half_win); i1 = min(len(ecg), idx+half_win+1)
    if i1-i0 <= 1: 
        return idx
    return i0 + np.argmax(ecg[i0:i1])

### 2.2 Algoritmo de Detección QRS

In [ ]:
def detect_qrs_rpeaks(ecg, fs):
    """
    Returns r_peaks (sample indices), qrs_on (onsets), qrs_off (offsets).
    Method: bandpass(5–15 Hz) → derivative → square → MWI(150 ms) → adaptive threshold.
    """
    # Emphasize QRS bandwidth
    #qrs_bp = bandpass(ecg, fs, low=5.0, high=15.0, order=3)

    # PT-style preprocessing
    #der = np.append(np.diff(qrs_bp), 0)                    # derivative
    der = np.append(np.diff(ecg), 0)                    # derivative
    sq = der**2                                            # nonlinearity
    mwi = moving_average(sq, int(0.150*fs))                # ~150 ms integration

    # Adaptive threshold based on robust stats
    thr = np.median(mwi) + 1.5*np.median(np.abs(mwi - np.median(mwi)))  # median + 1.5*MAD

    # Refractory period ~200 ms to avoid double detections
    min_dist = int(0.2 * fs)
    pk_idx, _ = find_peaks(mwi, height=thr, distance=min_dist)

    # Refine each detection to true R on band-limited ECG (steeper & higher than raw)
    #r_peaks = np.array([_refine_peak(qrs_bp, p, half_win=int(0.05*fs)) for p in pk_idx], dtype=int)
    r_peaks = np.array([_refine_peak(ecg, p, half_win=int(0.05*fs)) for p in pk_idx], dtype=int)

    # QRS onset/offset from energy curve around each R
    qrs_on = []
    qrs_off = []
    half = int(0.12 * fs)  # ±120 ms search
    for r in r_peaks:
        i0 = max(0, r - half); i1 = min(len(mwi), r + half)
        seg = mwi[i0:i1]
        # define a local, conservative threshold
        loc_thr = np.median(seg)
        # onset: last index from r backwards where energy falls below local threshold
        on = i0 + np.where(seg[:(r-i0)][::-1] < loc_thr)[0]
        on = r - on[0] if len(on) else i0
        # offset: first index after r where energy falls below local threshold
        off = i0 + (np.where(seg[(r-i0):] < loc_thr)[0][0] if np.any(seg[(r-i0):] < loc_thr) else (i1 - i0 - 1))
        qrs_on.append(int(on))
        qrs_off.append(int(off))
    return r_peaks, np.array(qrs_on), np.array(qrs_off)
    